<a href="https://colab.research.google.com/github/Rabittitan/MC-Team7/blob/main/genre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Khai Báo Thư Viện


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



# Thông Tin Dữ liệu

## Truyền Vào Tệp Dữ Liệu

In [ ]:
df = pd.read_csv("/content/spotify_songs.csv")
df.head()

## Kiểm Tra Dữ Liệu và Làm Sạch

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#xu ly du lieu null
df_cleaned = df.dropna()

print(df_cleaned.isnull().sum())

In [ ]:
# Xóa các cột cụ không cần thiết cho mô hình đánh giá trực tiếp trên df
columns_to_drop = ['track_id','playlist_name','playlist_subgenre' ,'track_name', 'track_artist', 'track_popularity', 'track_album_id', 'track_album_name', 'track_album_release_date', 'playlist_id']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Kiểm tra lại dữ liệu sau khi xóa các cột
df.head()


In [ ]:
df['playlist_genre'].value_counts()

chọn playlist_genre là target vì vậy cần phải chuyển string sang int

---

thực hiện việc ánh xạ (mapping) giữa các giá trị chuỗi trong cột 'playlist_genre' của DataFrame (df) và các giá trị số tương ứng theo một bảng ánh xạ được định nghĩa trước đó


In [ ]:
 # Định nghĩa mapping giữa chuỗi và số
genre_mapping = {'pop': 0, 'rap': 1, 'rock': 2, 'latin': 3, 'r&b': 4, 'edm': 5}

# Áp dụng map cho cột 'playlist_genre'
df['playlist_genre'] = df['playlist_genre'].map(genre_mapping)

# Kiểm tra lại kiểu dữ liệu
print(df['playlist_genre'].dtype)


In [ ]:

# Kiểm tra sự đồng đều dữ liệu
plt.figure(figsize=(10, 6))
sns.countplot(x='playlist_genre', data=df, palette='viridis')

# Thêm tên thể loại vào biểu đồ
plt.title('Phân phối của các lớp')
plt.xlabel('Thể loại nhạc')
plt.ylabel('Số lượng mẫu')

# Lấy tên thể loại từ từ điển ngược lại
genre_reverse_mapping = {v: k for k, v in genre_mapping.items()}
labels = [genre_reverse_mapping[i] for i in range(len(genre_mapping))]

# Hiển thị tên thể loại trên trục x
plt.xticks(ticks=range(len(genre_mapping)), labels=labels)
plt.show()

In [ ]:
# thể hiện sự tương quan của genre với các thuộc tính khác
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

# Mô Hình Học Máy

## Chuẩn Bị Dữ Liệu Cho Mô Hình

In [ ]:

X = df.drop('playlist_genre', axis=1)
y = df['playlist_genre']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Chuẩn hóa dữ liệu nếu cần
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Mô Hình RandomForest


In [ ]:

# Tạo mô hình RandomForestClassifier với số lượng cây là 100
RF_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình trên tập huấn luyện
RF_model.fit(X_train_scaled, y_train)

# Dự đoán trên tập kiểm tra
y_pred = RF_model.predict(X_test_scaled)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Random Forest Accuracy: {accuracy}')
print(f'Random Forest Confusion Matrix:\n{conf_matrix}')
print(classification_report(y_test, y_pred))

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# Get the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Mô Hình Decision Tree

In [ ]:
# Xây dựng và huấn luyện mô hình Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

# Đánh giá mô hình Decision Tree
dt_accuracy = accuracy_score(y_test, dt_pred)
dt_conf_matrix = confusion_matrix(y_test, dt_pred)

print(f'Decision Tree Accuracy: {dt_accuracy}')
print(f'Decision Tree Confusion Matrix:\n{dt_conf_matrix}')

## Mô Hình Gradient Boosting

In [ ]:
# Huấn luyện mô hình Gradient Boosting
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

gb_accuracy = accuracy_score(y_test, gb_pred)
gb_conf_matrix = confusion_matrix(y_test, gb_pred)

print(f'Gradient Boosting Accuracy: {gb_accuracy}')
print(f'Gradient Boosting Confusion Matrix:\n{gb_conf_matrix}')

## Mô Hình Logistic Regression

In [ ]:
# Huấn luyện mô hình Logistic Regression
logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(X_train_scaled, y_train)

# Dự đoán với mô hình Logistic Regression
logistic_pred = logistic_model.predict(X_test_scaled)

# Đánh giá mô hình Logistic Regression
logistic_accuracy = accuracy_score(y_test, logistic_pred)
logistic_conf_matrix = confusion_matrix(y_test, logistic_pred)

print(f'Logistic Regression Accuracy: {logistic_accuracy}')
print(f'Logistic Regression Confusion Matrix:\n{logistic_conf_matrix}')

## So Sánh hiệu suất giữa các mô hình

In [ ]:
# Độ chính xác của từng mô hình
accuracy_scores = [gb_accuracy, dt_accuracy, accuracy,logistic_accuracy]

# Tên của từng mô hình
model_names = ['Gradient Boosting', 'Decision Tree', 'Random Forest','Logistic Regression']

# Vẽ biểu đồ cột
plt.bar(model_names, accuracy_scores, color=['blue', 'green', 'orange','red'])
plt.xlabel('Mô hình')
plt.ylabel('Độ chính xác')
plt.title('So sánh độ chính xác của các mô hình')
plt.ylim(0, 1)  # Điều chỉnh phạm vi trục y nếu cần thiết
plt.show()


## Tối Ưu Hiệu suất

### Tối ưu RandomForest

In [ ]:
# Định nghĩa các giá trị cần thử nghiệm
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30]
}

# Tạo mô hình Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Tạo đối tượng GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')

# Tiến hành tìm kiếm trên tập huấn luyện
grid_search.fit(X_train_scaled, y_train)

# Lấy ra các tham số tối ưu
best_params = grid_search.best_params_

# In ra tham số tối ưu
print("Best Parameters:", best_params)

In [ ]:
# Sử dụng tham số tối ưu từ GridSearch
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']

# Tạo mô hình Random Forest với tham số tối ưu
optimal_RF_model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, random_state=42)

# Huấn luyện lại mô hình trên tập huấn luyện
optimal_RF_model.fit(X_train_scaled, y_train)

# Dự đoán trên tập kiểm tra
y_pred_optimal = optimal_RF_model.predict(X_test_scaled)

# Đánh giá mô hình với tham số tối ưu
accuracy_optimal = accuracy_score(y_test, y_pred_optimal)
conf_matrix_optimal = confusion_matrix(y_test, y_pred_optimal)

# In kết quả
print(f'Optimal Random Forest Accuracy: {accuracy_optimal}')
print(f'Optimal Random Forest Confusion Matrix:\n{conf_matrix_optimal}')
print(classification_report(y_test, y_pred_optimal))


### Tối ưu Decision Tree



In [ ]:
# Định nghĩa các giá trị cần thử nghiệm
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Tạo mô hình Decision Tree
dt_model = DecisionTreeClassifier()

# Sử dụng GridSearchCV để tìm siêu tham số tối ưu
grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra các giá trị tối ưu
print("Best Parameters:", grid_search.best_params_)

# Huấn luyện mô hình với các giá trị tối ưu
optimal_dt_model = grid_search.best_estimator_
optimal_dt_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
optimal_dt_pred = optimal_dt_model.predict(X_test)

# Đánh giá mô hình Decision Tree tối ưu
optimal_dt_accuracy = accuracy_score(y_test, optimal_dt_pred)
optimal_dt_conf_matrix = confusion_matrix(y_test, optimal_dt_pred)

print(f'Optimal Decision Tree Accuracy: {optimal_dt_accuracy}')
print(f'Optimal Decision Tree Confusion Matrix:\n{optimal_dt_conf_matrix}')

### Tối ưu Gradient Boosting

In [ ]:
aram_grid_gb = {
    'n_estimators': [50],
    'learning_rate': [0.01],
    'max_depth': [3],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
}

# Tạo mô hình Gradient Boosting
gb_model = GradientBoostingClassifier()

# Sử dụng GridSearchCV để tìm siêu tham số tối ưu
grid_search_gb = GridSearchCV(gb_model, param_grid_gb, cv=5, scoring='accuracy')
grid_search_gb.fit(X_train, y_train)

# In ra các giá trị tối ưu
print("Best Parameters for Gradient Boosting:", grid_search_gb.best_params_)

# Huấn luyện mô hình với các giá trị tối ưu
optimal_gb_model = grid_search_gb.best_estimator_
optimal_gb_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
optimal_gb_pred = optimal_gb_model.predict(X_test)

# Đánh giá mô hình Gradient Boosting tối ưu
optimal_gb_accuracy = accuracy_score(y_test, optimal_gb_pred)
optimal_gb_conf_matrix = confusion_matrix(y_test, optimal_gb_pred)

print(f'Optimal Gradient Boosting Accuracy: {optimal_gb_accuracy}')
print(f'Optimal Gradient Boosting Confusion Matrix:\n{optimal_gb_conf_matrix}')

### Tối ưu logistic Regression


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Định nghĩa mô hình Logistic Regression
logistic_model = LogisticRegression(random_state=42, max_iter=1000)

# Đặt các giá trị để thử nghiệm
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# Sử dụng Grid Search để tìm kiếm giá trị tốt nhất cho các tham số
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# In ra giá trị tốt nhất và tham số tương ứng
print("Best Parameters:", grid_search.best_params_)

# Dự đoán với mô hình đã được tối ưu hóa
logistic_optimized_model = grid_search.best_estimator_
logistic_optimized_pred = logistic_optimized_model.predict(X_test_scaled)

# Đánh giá mô hình đã được tối ưu hóa
logistic_optimized_accuracy = accuracy_score(y_test, logistic_optimized_pred)
logistic_optimized_conf_matrix = confusion_matrix(y_test, logistic_optimized_pred)

print(f'Optimized Logistic Regression Accuracy: {logistic_optimized_accuracy}')
print(f'Optimized Logistic Regression Confusion Matrix:\n{logistic_optimized_conf_matrix}')


In [ ]:
# Các giá trị hiệu suất của các mô hình
accuracies = [accuracy_optimal, optimal_dt_accuracy, optimal_gb_accuracy, logistic_optimized_accuracy]
models = ['Random Forest', 'Decision Tree', 'Gradient Boosting', 'Logistic Regression']

# Vẽ bar chart
plt.figure(figsize=(10, 6))
plt.bar(models, accuracies, color=['blue', 'orange', 'green', 'red'])
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 cho phù hợp với tỷ lệ độ chính xác
plt.title('Comparison of Model Performances')
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.show()